<a href="https://colab.research.google.com/github/HelmchenLabSoftware/Cascade/blob/master/Demo%20scripts/Calibrated_spike_inference_with_Cascade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CASCADE

## Calibrated spike inference from calcium imaging data using deep networks
Written and maintained by [Peter Rupprecht](https://github.com/PTRRupprecht) and [Adrian Hoffmann](https://github.com/AdrianHoffmann) from the [Helmchen Lab](https://www.hifo.uzh.ch/en/research/helmchen.html).
The project started as a collaboration of the Helmchen Lab and the [Friedrich Lab](https://www.fmi.ch/research-groups/groupleader.html?group=119). Feedback goes to [Peter Rupprecht](mailto:p.t.r.rupprecht+cascade@gmail.com).

---

Adapted by Tony Hyun Kim, Sept 2021.

In [1]:
import os

path_to_cascade = 'C:/Users/herrin/Documents/Cascade'
os.chdir(path_to_cascade)
print('Current working directory:', os.getcwd())

Current working directory: C:\Users\herrin\Documents\Cascade


In [2]:
#@markdown Downloads packages from public repository, and packages from Cascade.

!pip install ruamel.yaml

# standard python packages
import os, warnings
import glob
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import ruamel.yaml as yaml
import h5py

# cascade2p packages, imported from the downloaded Github repository
from cascade2p import checks
checks.check_packages()
from cascade2p import cascade # local folder
from cascade2p.utils import plot_dFF_traces, plot_noise_level_distribution, plot_noise_matched_ground_truth

# THK: Loads traces from REC files
def load_traces(file_path):
    """Load traces from REC files with output shape [num_neuron x num_frames]"""
    f = h5py.File(file_path, 'r')
    
    traces = np.array(f['traces'])
    info = f['info']
    frame_rate = info['fps'][()].item()
    
    return traces, frame_rate

	YAML reader installed (version 0.16.12).


Using TensorFlow backend.


	Keras installed (version 2.3.1).
	Tensorflow installed (version 2.1.0).


In [3]:
path_to_rec = "Y:/Downloads/oh12-0123-ctx/union_15hz/dff"

path_to_rec = os.path.join(path_to_rec, "rec_*.mat")
rec_file = glob.glob(path_to_rec)[0]
print('Located REC file:', rec_file)

Located REC file: Y:/Downloads/oh12-0123-ctx/union_15hz/dff\rec_210905-151523.mat


In [4]:
#@markdown If you are testing the script, you can leave everything unchanged. If you want to apply the algorithm to your own data, you have to upload your data first. The paragraph above tells you how to format and name the file. You can do this by clicking on the **folder symbol ("Files")** on the left side of the Colaboratory notebook. Next, indicate the path of the uploaded file in the variable **`example_file`**. Finally, indicate the sampling rate of your recordings in the variable **`frame_rate`**.

try:

  traces, frame_rate = load_traces(rec_file)
  print(rec_file)
  print('Number of neurons in dataset:', traces.shape[0])
  print('Number of timepoints in dataset:', traces.shape[1])
  print('Frame rate:', frame_rate)

except Exception as e:

  print('Error message: '+str(e))
  

Y:/Downloads/oh12-0123-ctx/union_15hz/dff\rec_210905-151523.mat
Number of neurons in dataset: 39
Number of timepoints in dataset: 16000
Frame rate: 15.0


In [5]:
model_name = "Global_EXC_15Hz_smoothing200ms" #@param {type:"string"}

spike_probs = cascade.predict( model_name, traces )

# save as mat file
folder = os.path.dirname(rec_file)
file_name = 'cascade_' + model_name
save_path = os.path.join(folder, file_name)

sio.savemat(save_path+'.mat', {'spike_probs': np.transpose(spike_probs), 'model_name': model_name, 'rec_file': rec_file})



 
The selected model was trained on 18 datasets, with 5 ensembles for each noise level, at a sampling rate of 15Hz, with a resampled ground truth that was smoothed with a Gaussian kernel of a standard deviation of 200 milliseconds. 
 

Loaded model was trained at frame rate 15 Hz
Given argument traces contains 39 neurons and 16000 frames.
Noise levels (mean, std; in standard units): 1.37, 0.19

Predictions for noise level 2:
	... ensemble 0
624000/624000 [==============================] - 7s 11us/sample
	... ensemble 1
624000/624000 [==============================] - 7s 12us/sample
	... ensemble 2
624000/624000 [==============================] - 7s 11us/sample
	... ensemble 3
624000/624000 [==============================] - 7s 12us/sample
	... ensemble 4
624000/624000 [==============================] - 7s 11us/sample

Predictions for noise level 3:
	No neurons for this noise level

Predictions for noise level 4:
	No neurons for this noise level

Predictions for noise level 5:
	No neur